# Raw data
--------------
## first check the raw data

In [4]:
# check how many lines
! wc -l data/emojitweets-01-04-2018.txt

18866900 data/emojitweets-01-04-2018.txt


In [2]:
# check how many words
! wc -w data/emojitweets-01-04-2018.txt

209430000 data/emojitweets-01-04-2018.txt


In [3]:
# On average the sentense are have 11 words
209430000/18866900

11.100392751326398

## Train - test - validation split

In [6]:
# split on 80-10-10
18866900 * 0.8, 18866900 * 0.1, 18866900 * 0.2

(15093520.0, 1886690.0, 3773380.0)

In [8]:
! head -15093520 data/emojitweets-01-04-2018.txt >>emoji_train.txt


In [14]:
! (tail -3773380 data/emojitweets-01-04-2018.txt | head -1886690) >> emoji_val.txt

tail: error writing 'standard output': Broken pipe


In [10]:
! tail -1886690 data/emojitweets-01-04-2018.txt >> emoji_test.txt

In [1]:
# generate small_sample
! head -1509352 data/emoji_train.txt >> data/emoji_ss.txt

# Word Vectorize

In [2]:
# Python program to generate word vectors using Word2Vec 
  
# importing all necessary modules 
from nltk.tokenize import sent_tokenize, word_tokenize 
import warnings 
  
warnings.filterwarnings(action = 'ignore') 
  
import gensim 
from gensim.models import Word2Vec

import emoji
import regex

import pandas as pd
import numpy as np
import pickle


In [2]:
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer


In [49]:
# sample = open("emoji_1M.txt", "r") 
sample = open("data/emoji_ss.txt", "r") 
s = sample.read() 
  
# Replaces escape character with space 
f = s.replace("\n", " ") 

In [50]:
# replace all 
notes = [",", ".", "/'", "(" ,")", "'", '/"', ":", ";", "/", "(", ")", "“", "”", "-", "+", "#", "…", "!"]
for note in notes:
    f = f.replace(note, "")

  

In [1]:

# FULLLLLLLLLLLLL data
sample = open("data/emoji_train.txt", "r") 
s = sample.read() 
  
# Replaces escape character with space 
f = s.replace("\n", " ") 
# replace all 
notes = [",", ".", "/'", "(" ,")", "'", '/"', ":", ";", "/", "(", ")", "“", "”", "-", "+", "#", "…", "!"]
for note in notes:
    f = f.replace(note, "")


In [5]:
  
data = [] 
# iterate through each sentence in the file 
for i in sent_tokenize(f): 
    temp = [] 
      
    # tokenize the sentence into words 
    for j in word_tokenize(i): 
        temp.append(j.lower()) 
  
    data.append(temp) 

In [44]:
len(f)

84618238

In [114]:
# move to helper
def split_count(text):

    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)

    return emoji_list

run for stemmer


In [14]:
# import stopwords
stop = set(stopwords.words('english'))
# remove stop words and snowball stemmer
out = [[word for word in words if word not in stop] for words in data]
snowball = SnowballStemmer('english')
docs_snowball = [[snowball.stem(word) for word in words] for words in out]

In [142]:
len(docs_snowball)

79777

In [144]:
len(docs_snowball[0])

266

save emoji list

In [10]:
# emoji_list = list(emoji_set)
# with open("emoji_list.csv", 'w', newline='') as myfile:
#      w = csv.writer(myfile, quoting=csv.QUOTE_ALL)
#      w.writerow(emoji_list)

4358

In [ ]:
# with open("emoji_list.csv", 'r', newline='') as myfile:
#      emoji_list = csv.reader(myfile, quoting=csv.QUOTE_ALL)

# CBOW model

In [6]:
# Create CBOW model 
model1 = gensim.models.Word2Vec(data, 
                                min_count = 10,  
                                size = 300, 
                                window = 16,
                                workers= 8) 

In [7]:
# # Save CBOW model
# import pickle
# with open('cbow_ss.pkl', 'wb') as f:
#     pickle.dump(model1, f)

In [ ]:
# load CBOW model
import pickle
with open('cbow.pkl', 'rb') as f2:
    model1_1 = pickle.load(f2)
# print(model1_1 == model1)

In [3]:
val_df = pd.read_csv("data/emoji_val.2csv",index_col=0)
emoji_set = emoji_list = val_df["emoji"].values.tolist()

In [13]:
# move to helper
def emoji_prodictor(word, model):
    sim = model.most_similar(word,topn=75)
    for i in sim:
        if i[0] in emoji_list:
            return i[0]
    return word

In [21]:
# move to helper
def word_pipeline(sentense):
    out = [word for word in sentense.lower().split(" ") if word not in stop]
    out = [snowball.stem(word) for word in out]
    return out
    
    
s1 = "let's go grocery and buy some apple"
word_pipeline(s1)

['let', 'go', 'groceri', 'buy', 'appl']

In [13]:
s1 = "let's go grocery and buy some apple"
for w in word_pipeline(s1):
    print(emoji_prodictor(w, model1))


🤷‍♀️
🤦🏾‍♀️
🛒
💸
👐🏽


In [20]:
emoji_prodictor("today")

'👌🏼'

# skip-gram model

In [10]:
# Create skip-gram model 
model2 = gensim.models.Word2Vec(data, 
                                min_count = 10,  
                                size = 300, 
                                window = 16,
                                sg = 1,
                                workers=8) 

In [11]:
# # Save skip-gram model
# import pickle
# with open('sg_ss.pkl', 'wb') as f:
#     pickle.dump(model2, f)


In [ ]:
# load skip-gram model
import pickle
with open('sg_ss.pkl', 'rb') as f2:
    model2_2 = pickle.load(f2)
print(model2_2 == model2)

In [125]:
s1 = "let's go grocery and buy some apples"
for w in word_pipeline(s1):
    print(emoji_prodictor(w, model2))


😂
😂
😀
buy
🍎


In [14]:
model2.most_similar("star",topn=20)

[('now🎮', 0.7090588808059692),
 ('puzzle', 0.6862722635269165),
 ('now~🎮', 0.6172725558280945),
 ('google', 0.5657158493995667),
 ('on💡', 0.5306732058525085),
 ('stylish', 0.5125507712364197),
 ('missions', 0.5065516233444214),
 ('all😆', 0.4989703595638275),
 ('🎰', 0.4774476885795593),
 ('nightmare⚡️', 0.4716939926147461),
 ('app', 0.46512648463249207),
 ('fame🌟', 0.4634802043437958),
 ('days‼️', 0.4631275534629822),
 ('go‼️', 0.4629686772823334),
 ('trek', 0.4612944722175598),
 ('🔥who', 0.45808354020118713),
 ('shining', 0.4320569932460785),
 ('ever~🌟', 0.42865443229675293),
 ('🙋', 0.42692843079566956),
 ('play', 0.4189406931400299)]


# Scoring

In [17]:
# load the ground-truth
with open('val_dict2.pickle', 'rb') as handle:
    val_dict = pickle.load(handle)


In [18]:
# # define score function with stemmer and trim stop words
# def word2vec_score(val_dict, model):
#     total_words = 0
#     total_predicts = 0
#     correct = 0
#     words = []
#     word_vectors = model.wv
#     for word in val_dict:
# #         print(word)
#         if word_pipeline(word):
#             w = word_pipeline(word)[0]
#             words.append(w)
#             if w in word_vectors:
#                 total_words += 1
#                 if emoji_prodictor(w, model) in val_dict[word]:
#                     correct += 1
#                     total_predicts += 1
#                 elif emoji_prodictor(w, model) in emoji_set:
#                     total_predicts += 1
#     return correct/total_words, correct/total_predicts, words
            

In [15]:
# define score function 
def word2vec_score(val_dict, model):
    total_words = 0
    total_predicts = 0
    correct = 0
    words = []
    word_vectors = model.wv
    for word in val_dict:
#         print(word)
        if word:
            w = word.lower()
            words.append(w)
            if w in word_vectors:
                total_words += 1
                if emoji_prodictor(w, model) in val_dict[word]:
                    correct += 1
                    total_predicts += 1
                elif emoji_prodictor(w, model) in emoji_set:
                    total_predicts += 1
    return correct/total_words, correct/total_predicts, words
            

In [20]:
score1 = word2vec_score(val_dict, model1)

In [21]:
score2 = word2vec_score(val_dict, model2)

In [ ]:
# baseline for ss, 0.04464

In [22]:
score1[0:2],score2[0:2]

((0.16235446313065977, 0.21978984238178634),
 (0.2128072445019405, 0.2634107285828663))

--------------------
Weighted score

In [83]:
def word2vec_weighted_score(val_dict, model,emoji_frequency):
    total_words = 0
    total_weighted = 0
    correct_weighted = 0
#     words = []
    word_vectors = model.wv
    for word in val_dict:
        if word:
            w = word.lower()
#             words.append(w)
            if w in word_vectors:
                total_words += 1
                prediction = emoji_prodictor(w, model)
                if prediction in val_dict[word]:
                    correct_weighted += emoji_frequency[prediction]
                    total_weighted += emoji_frequency[prediction]
                elif prediction in emoji_set:
                    total_weighted += emoji_frequency[prediction]
    return correct_weighted/total_weighted

In [84]:
word2vec_weighted_score(val_dict, model1, emoji_frequency)    # CBOW good at predicting more frequent word (high weight)

0.267386829730014

In [85]:
word2vec_weighted_score(val_dict, model2, emoji_frequency)    # Skip-gram good at predicting less frequent word (low weight)

0.06241611360445485

--------------------
more restriction on prediction

In [78]:
# new predictor with higher requirement to predict
def emoji_prodictor(word, model):
    sim = model.most_similar(word,topn=40)
    for i in sim:
        if i[0] in emoji_list and i[1] >= 0.55:
            return i[0]
    return word

In [79]:
score1 = word2vec_score(val_dict, model1)
score2 = word2vec_score(val_dict, model2)

In [80]:
# score for CBOW and skipgram for SS WITH min_count = 10,  size = 300, window = 16 and new notes / predictor sim>=0.55
# We can see improvement on accuracy but a drop in coverage
score1[0:2],score2[0:2]

((0.08711885500933417, 0.5072463768115942),
 (0.06782825140012445, 0.5647668393782384))

In [84]:
word2vec_weighted_score(val_dict, model1, emoji_frequency)

0.5681407491990289

In [85]:
word2vec_weighted_score(val_dict, model2, emoji_frequency)

0.12596177586164578

----------------------------------------------------
Model tuning history

In [ ]:
# FULL TRAIN
# score for CBOW and skipgram for SS WITH min_count = 10,  size = 300, window = 16 and no stemmer

# score1[:2] (0.2809917355371901, 0.4602888086642599)
# score2[:2] (0.30413223140495865, 0.4302416212003118)

In [108]:
# score for CBOW and skipgram for SS WITH min_count = 10,  size = 300, window = 16 and no stemmer
# Looks like stemmer does not help much
score1[0:2],score2[0:2]

((0.1630012936610608, 0.2202797202797203),
 (0.21798188874514876, 0.27897350993377484))

In [76]:
# Weighted score for CBOW and skipgram for SS WITH min_count = 10,  size = 300, window = 16 and new notes
word2vec_weighted_score(val_dict, model1, emoji_frequency)

0.256142944437114

In [77]:
word2vec_weighted_score(val_dict, model2, emoji_frequency)

0.09182329105692395

In [27]:
# score for CBOW and skipgram for SS WITH min_count = 10,  size = 300, window = 16 and new notes
score1[0:2],score2[0:2]

((0.16490354698195395, 0.19188993482983346),
 (0.22028624766645924, 0.2512420156139106))

In [123]:
# score for CBOW and skipgram for SS WITH min_count = 10,  size = 300, window = 16 and new notes
score1[0:2],score2[0:2]

((0.16936488169364883, 0.19470293486041518),
 (0.22478206724782068, 0.2565742714996446))

In [104]:
# score for CBOW and skipgram for SS WITH min_count = 10,  size = 300, window = 16
score1[0:2],score2[0:2]

((0.16084788029925187, 0.19068736141906872),
 (0.21571072319201995, 0.25181950509461426))

In [82]:
# score for CBOW and skipgram for SS WITH min_count = 10,  size = 300, window = 20
score1[0:2],score2[0:2]

((0.15149625935162095, 0.17711370262390672),
 (0.21571072319201995, 0.24162011173184358))

In [77]:
# score for CBOW and skipgram for SS WITH min_count = 10,  size = 150, window = 10
score1[0:2],score2[0:2]

((0.1589775561097257, 0.19480519480519481),
 (0.18952618453865336, 0.22943396226415094))

In [72]:
# score for CBOW and skipgram for SS WITH min_count = 10,  size = 300, window = 10
score1[0:2],score2[0:2]

((0.17206982543640897, 0.21052631578947367),
 (0.2013715710723192, 0.2492283950617284))

In [66]:
# score for CBOW and skipgram for SS WITH min_count = 10,  size = 500, window = 10
score1[0:2],score2[0:2]

((0.16521197007481297, 0.1990984222389181),
 (0.20199501246882792, 0.2577565632458234))

In [61]:
# score for CBOW and skipgram for SS WITH min_count = 5,  size = 500, window = 8
score1[0:2],score2[0:2]

((0.16390532544378697, 0.22124600638977635),
 (0.17810650887573964, 0.2872137404580153))

In [55]:
# score for CBOW and skipgram for SS WITH min_count = 10,  size = 500, window = 8
score1[0:2],score2[0:2]

((0.16209476309226933, 0.19667170953101362),
 (0.19887780548628428, 0.2570507655116841))

In [50]:
# score for CBOW and skipgram for SS WITH min_count = 10,  size = 500, window = 3
score1[0:2],score2[0:2]

((0.14526184538653367, 0.19082719082719082),
 (0.1502493765586035, 0.21929026387625114))

In [56]:
# # score for CBOW and skipgram for small sample with SnowballStemmer
# score1[0:2],score2[0:2]

((0.14964788732394366, 0.18695014662756598),
 (0.15307262569832403, 0.23539518900343642))

____________________________________________

In [ ]:
# baseline of prediction

In [4]:
emoji_list[::200]

['😀',
 '💙',
 '🙌🏻',
 '👩🏾\u200d\U0001f9b3',
 '💁🏿\u200d♂',
 '\U0001f9d1\u200d🎓',
 '\U0001f9d1🏼\u200d🎤',
 '💂🏿\u200d♀',
 '\U0001f9d1🏻\u200d🎄',
 '\U0001f9dd🏾\u200d♂',
 '👨🏻\u200d\U0001f9bc',
 '🏄\u200d♂',
 '🚵🏿\u200d♂',
 '\U0001f9d1🏾\u200d🤝\u200d\U0001f9d1🏿',
 '\U0001f9a6',
 '🍾',
 '🕑',
 '👘',
 '🔓',
 '⏪',
 '🏳',
 '🇵🇾']

In [5]:
from collections import Counter

In [ ]:
# word frequency
c = Counter(f.split(" "))
# c.most_common()

In [ ]:
# emoji_freq
emoji_frequency = {}

total = 0
for emoji in emoji_list:
    emoji_frequency[emoji] = c[emoji]
    total += c[emoji]

In [65]:
emoji_frequency2 = {}
for key, value in emoji_frequency.items():
    emoji_frequency2[key] = value/total

In [54]:
sort_freq = {k: v for k, v in sorted(emoji_frequency.items(), key=lambda item: item[1], reverse=True)}

In [64]:
# baseline
69149/total

0.04464584326015571

In [61]:
sort_freq

{'😂': 69149,
 '🔥': 33328,
 '❤️': 27982,
 '😭': 25433,
 '😍': 22820,
 '👉': 17657,
 '💕': 12445,
 '😊': 12041,
 '✨': 11794,
 '🤔': 10607,
 '👀': 10602,
 '😩': 10214,
 '❤': 10076,
 '💜': 8118,
 '🙄': 7977,
 '✅': 7664,
 '🚨': 7606,
 '💯': 7535,
 '🔔': 7372,
 '💙': 7344,
 '🤣': 7226,
 '😘': 7138,
 '💀': 7133,
 '😎': 6958,
 '😉': 6916,
 '💎': 6611,
 '🙌': 6543,
 '🙏': 6248,
 '©': 5899,
 '🎉': 5817,
 '💖': 5794,
 '🏆': 5700,
 '👏': 5532,
 '♨️': 5469,
 '💥': 5386,
 '😁': 5300,
 '📹': 5130,
 '🙃': 4995,
 '🌸': 4923,
 '😢': 4904,
 '👍': 4892,
 '📷': 4632,
 '🇺🇸': 4576,
 '🙏🏻': 4513,
 '➡️': 4484,
 '💛': 4405,
 '😳': 4359,
 '😅': 4335,
 '🗣': 4264,
 '👈': 4138,
 '🤗': 4078,
 '🎶': 3946,
 '👑': 3906,
 '😔': 3883,
 '💋': 3846,
 '💚': 3809,
 '😏': 3805,
 '☺️': 3799,
 '💗': 3671,
 '📸': 3641,
 '⏬': 3547,
 '♥️': 3539,
 '💫': 3525,
 '🔗': 3486,
 '🤷🏽\u200d♀️': 3421,
 '💔': 3340,
 '🖤': 3287,
 '👌': 3287,
 '😌': 3283,
 '💪': 3282,
 '🌟': 3207,
 '👇': 3186,
 '🎥': 3134,
 '🙂': 3065,
 '😒': 3046,
 '➡': 3039,
 '😱': 3012,
 '✔️': 2994,
 '💞': 2985,
 '‼️': 2970,
 '🔴': 292

In [56]:
non_zero_count = 0
count = 0
for key, value in sort_freq.items():
    count += 1
    if value > 0:
        non_zero_count += 1
count, non_zero_count

(4168, 2446)

In [58]:
list(sort_freq.keys())

['😂',
 '🔥',
 '❤️',
 '😭',
 '😍',
 '👉',
 '💕',
 '😊',
 '✨',
 '🤔',
 '👀',
 '😩',
 '❤',
 '💜',
 '🙄',
 '✅',
 '🚨',
 '💯',
 '🔔',
 '💙',
 '🤣',
 '😘',
 '💀',
 '😎',
 '😉',
 '💎',
 '🙌',
 '🙏',
 '©',
 '🎉',
 '💖',
 '🏆',
 '👏',
 '♨️',
 '💥',
 '😁',
 '📹',
 '🙃',
 '🌸',
 '😢',
 '👍',
 '📷',
 '🇺🇸',
 '🙏🏻',
 '➡️',
 '💛',
 '😳',
 '😅',
 '🗣',
 '👈',
 '🤗',
 '🎶',
 '👑',
 '😔',
 '💋',
 '💚',
 '😏',
 '☺️',
 '💗',
 '📸',
 '⏬',
 '♥️',
 '💫',
 '🔗',
 '🤷🏽\u200d♀️',
 '💔',
 '🖤',
 '👌',
 '😌',
 '💪',
 '🌟',
 '👇',
 '🎥',
 '🙂',
 '😒',
 '➡',
 '😱',
 '✔️',
 '💞',
 '‼️',
 '🔴',
 '😈',
 '🌹',
 '🤦🏽\u200d♀️',
 '😀',
 '♥',
 '🔜',
 '⚠',
 '😜',
 '😆',
 '⚡',
 '☀️',
 '😋',
 '😄',
 '💓',
 '🌑',
 '✔',
 '🌒',
 '😤',
 '🌓',
 '🤧',
 '🐰',
 '\U0001f92a',
 '🎁',
 '📺',
 '💘',
 '😞',
 '😬',
 '🌕',
 '❌',
 '😪',
 '🚀',
 '🌘',
 '🌖',
 '🌗',
 '🌔',
 '⏰',
 '🌍',
 '☹️',
 '😇',
 '🙏🏾',
 '😐',
 '😴',
 '💦',
 '🙏🏽',
 '🙈',
 '🐣',
 '🌞',
 '📣',
 '😫',
 '⚠️',
 '🤷🏽\u200d♂️',
 '👅',
 '💰',
 '😕',
 '🏨',
 '👊',
 '🌎',
 '🤤',
 '🤷🏾\u200d♂️',
 '🌻',
 '📎',
 '👏🏻',
 '⬇️',
 '😃',
 '🍀',
 '🗳',
 '🔹',
 '📢',
 '🙏🏼',
 '🤷🏾\u200d♀️',
 '\U0001f929',
 '😑',
 '

In [150]:
model2.most_similar('😀',topn=20)

[('👍', 0.5708750486373901),
 ('😉', 0.5507802963256836),
 ('😁', 0.5429814457893372),
 ('⭐️i', 0.5380228757858276),
 ('awesom', 0.5341282486915588),
 ('😅', 0.5143481492996216),
 ('🤗', 0.5049486756324768),
 ('😏', 0.503909707069397),
 ('😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂', 0.4923427402973175),
 ('😘', 0.4906120002269745),
 ('wonder', 0.48665982484817505),
 ('goan', 0.4851222038269043),
 ('jeanett', 0.47811296582221985),
 ('skyrim', 0.4774525463581085),
 ('🙀🙀🙀', 0.47732964158058167),
 ('😊', 0.47706007957458496),
 ('👍👊', 0.4713195264339447),
 ('carsss', 0.47071343660354614),
 ('😍thank', 0.4696727693080902),
 ('💞🌺💕', 0.4684681296348572)]

In [97]:
emoji_prodictor('happi', model2)

False

In [ ]:
def word_pipeline(sentense):
    out = [word for word in sentense.lower().split(" ") if word not in stop]
    out = [snowball.stem(word) for word in out]
    return out

In [115]:
sentense = "under"
[word for word in sentense.lower().split(" ") if word not in stop]

[]

In [116]:
word_pipeline(sentense)model2

[]

In [65]:
emoji_prodictor('us', model1)

'😀'

In [71]:
emoji_prodictor('nyc', model1)

'🚌'

-----------------------
# Ensemble model

In [23]:
# original scores
score1[0:2],score2[0:2]

((0.16235446313065977, 0.21978984238178634),
 (0.2128072445019405, 0.2634107285828663))

In [24]:
word="happy"

In [30]:
sim = model1.most_similar(word,topn=75)
sim.extend(model2.most_similar(word,topn=75))
sim = pd.DataFrame(sim).sort_values(by=1, ascending=False).set_index(0).to_dict()[1]

In [40]:
sim = pd.DataFrame(sim).sort_values(by=1, ascending=False).set_index(0).to_dict()[1]

In [41]:
# ensemble by combine the correlations list
def ensemble_emoji_prodictor(word, model1, model2):
    sim = model1.most_similar(word,topn=75)
    sim.extend(model2.most_similar(word,topn=75))
    sim = pd.DataFrame(sim).sort_values(by=1, ascending=False).set_index(0).to_dict()[1]
    
    for i in sim:
        if i in emoji_list:
            return i
               
    return word

In [44]:
# define score function 
def word2vec_score_emsemble(val_dict,model1, model2):
    total_words = 0
    total_predicts = 0
    correct = 0
    words = []
    word_vectors = model1.wv
    for word in val_dict:
#         print(word)
        if word:
            w = word.lower()
            words.append(w)
            if w in word_vectors:
                total_words += 1
                pred = ensemble_emoji_prodictor(w, model1, model2)
                if pred in val_dict[word]:
                    correct += 1
                    total_predicts += 1
                elif pred in emoji_set:
                    total_predicts += 1
    return correct/total_words, correct/total_predicts, words
            

In [46]:
word2vec_score_emsemble(val_dict,model1, model2)[:2]

(0.20633893919793014, 0.22592067988668554)

ensemble the correlation doesnot work


In [77]:
# ensemble by combine the correlations list
def ensemble_emoji_prodictor(word, model1, model2, thres):
    p1 = emoji_prodictor(word, model1)
    p2 = emoji_prodictor(word, model2)
    t = int(thres*non_zero_count)
    l1 = list(sort_freq.keys())[:t]
    l2 = list(sort_freq.keys())[t:]
    if p1 in l1:
        return p1
    else:
        return p2

In [62]:
# define score function 
def word2vec_score_emsemble(val_dict,model1, model2, thres):
    total_words = 0
    total_predicts = 0
    correct = 0
    words = []
    word_vectors = model1.wv
    for word in val_dict:
#         print(word)
        if word:
            w = word.lower()
            words.append(w)
            if w in word_vectors:
                total_words += 1
                pred = ensemble_emoji_prodictor(w, model1, model2, thres)
                if pred in val_dict[word]:
                    correct += 1
                    total_predicts += 1
                elif pred in emoji_set:
                    total_predicts += 1
    return correct/total_words, correct/total_predicts, words
            

In [74]:
word2vec_score_emsemble(val_dict,model1, model2, 0.5)[:2]

(0.17658473479948253, 0.2161520190023753)

In [75]:
thres_arr = np.linspace(0.05,0.95,19);thres_arr

array([0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 , 0.55,
       0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95])

In [76]:
# if p2 in l2
for thres in thres_arr:
    print(thres, word2vec_score_emsemble(val_dict,model1, model2, thres)[:2])

0.05 (0.21733505821474774, 0.24651504035216434)
0.1 (0.2166882276843467, 0.2501867064973861)
0.15 (0.2147477360931436, 0.25018839487565936)
0.2 (0.2089262613195343, 0.24506828528072838)
0.25 (0.20310478654592498, 0.2389649923896499)
0.3 (0.19728331177231564, 0.23318042813455658)
0.35 (0.18952134540750323, 0.22608024691358025)
0.39999999999999997 (0.18628719275549807, 0.223950233281493)
0.44999999999999996 (0.18305304010349288, 0.22092115534738485)
0.49999999999999994 (0.17658473479948253, 0.2161520190023753)
0.5499999999999999 (0.17270375161707632, 0.21445783132530122)
0.6 (0.17076326002587322, 0.21782178217821782)
0.65 (0.1649417852522639, 0.21982758620689655)
0.7 (0.1630012936610608, 0.2202797202797203)
0.75 (0.1630012936610608, 0.2204724409448819)
0.7999999999999999 (0.16235446313065977, 0.21978984238178634)
0.85 (0.16235446313065977, 0.21978984238178634)
0.9 (0.16235446313065977, 0.21978984238178634)
0.95 (0.16235446313065977, 0.21978984238178634)


In [79]:
# if p1 in l1
thres_arr = np.linspace(0.01,0.20,20)
for thres in thres_arr:
    print(thres, word2vec_score_emsemble(val_dict,model1, model2, thres)[:2])

0.01 (0.21410090556274256, 0.26501200960768617)
0.02 (0.21539456662354464, 0.266613290632506)
0.03 (0.21539456662354464, 0.26618705035971224)
0.04 (0.21539456662354464, 0.26618705035971224)
0.05 (0.21604139715394566, 0.266986410871303)
0.060000000000000005 (0.21798188874514876, 0.2687400318979266)
0.06999999999999999 (0.21798188874514876, 0.2685258964143426)
0.08 (0.21798188874514876, 0.2680986475735879)
0.09 (0.21733505821474774, 0.2670906200317965)
0.09999999999999999 (0.21733505821474774, 0.26666666666666666)
0.11 (0.21733505821474774, 0.26645519429024583)
0.12 (0.21733505821474774, 0.26645519429024583)
0.13 (0.2166882276843467, 0.2654516640253566)
0.14 (0.21733505821474774, 0.26540284360189575)
0.15000000000000002 (0.21733505821474774, 0.26498422712933756)
0.16 (0.21604139715394566, 0.2623723487824038)
0.17 (0.2166882276843467, 0.2619233776387803)
0.18000000000000002 (0.21539456662354464, 0.26015625)
0.19 (0.21604139715394566, 0.2595182595182595)
0.2 (0.2166882276843467, 0.26009316

In [ ]:
((0.16235446313065977, 0.21978984238178634),
 (0.2128072445019405, 0.2634107285828663))

In [86]:
word2vec_weighted_score(val_dict, model1, emoji_frequency)    # CBOW good at predicting more frequent word (high weight)

0.267386829730014

In [87]:
word2vec_weighted_score(val_dict, model2, emoji_frequency)

0.06241611360445485

In [89]:
def word2vec_weighted_score_ensemble(val_dict, model1, model2, thres, emoji_frequency):
    total_words = 0
    total_weighted = 0
    correct_weighted = 0
#     words = []
    word_vectors = model1.wv
    for word in val_dict:
        if word:
            w = word.lower()
#             words.append(w)
            if w in word_vectors:
                total_words += 1
                prediction = ensemble_emoji_prodictor(w, model1, model2, thres)
                if prediction in val_dict[word]:
                    correct_weighted += emoji_frequency[prediction]
                    total_weighted += emoji_frequency[prediction]
                elif prediction in emoji_set:
                    total_weighted += emoji_frequency[prediction]
    return correct_weighted/total_weighted

In [90]:
word2vec_weighted_score_ensemble(val_dict, model1, model2, 0.06, emoji_frequency)

0.09934862490553567

In [91]:
# if p1 in l1
thres_arr = np.linspace(0.01,0.20,20)
for thres in thres_arr:
    print(word2vec_weighted_score_ensemble(val_dict, model1, model2, thres, emoji_frequency))

0.08474481924748074
0.09006502653848159
0.09479537639495465
0.09471136964476934
0.09519971373942336
0.09934862490553567
0.10045718196569688
0.10347345581361085
0.10329134014169694
0.10327865628196051
0.10330389100633897
0.10388116539603176
0.10619495618015533
0.10655389009358177
0.11036054258964167
0.11017199038168167
0.11088114427920304
0.11100778048100647
0.11247291178759795
0.11263821026961353


In [96]:
s1 = "Head shoulders knees and toes knees and toes"
s2 = "And eyes and ears and mouth and mouth and nose"

In [98]:
for w in s1.lower().split(" "):
    print(emoji_prodictor(w, model1))
for w in s2.lower().split(" "):
    print(emoji_prodictor(w, model1))

head
shoulders
knees
😊
🍆
knees
😊
🍆
😊
eyes
😊
ears
😊
🍆
😊
🍆
😊
nose


In [99]:
for w in s1.lower().split(" "):
    print(emoji_prodictor(w, model2))
for w in s2.lower().split(" "):
    print(emoji_prodictor(w, model2))

head
💁🏾
🙇🏽‍♀️
😂
💅
🙇🏽‍♀️
😂
💅
😂
eyes
😂
ears
😂
🙇🏾‍♀️
😂
🙇🏾‍♀️
😂
nose


-------------------------
# RNN model

In [111]:
with open('emoji_100k.txt', 'r') as file:
    text = file.read().lower()
print('text length', len(text))
chars = sorted(list(set(text))) # getting all unique chars
print('total chars: ', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

text length 5835178
total chars:  2338


RNN model not working here since RNN requires a large X matrix, with 3 dimension as (num_of_lines * window_size * num_of_chars)
Since we have a very large num_of_char and lots of num_of_lines. It requires hundreds of gigabytes memory to run each for a small sample.

In [112]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

MemoryError: Unable to allocate 169. GiB for an array with shape (1945046, 40, 2338) and data type bool

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [ ]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [ ]:
from keras.callbacks import ModelCheckpoint

filepath = "weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss',
                             verbose=1, save_best_only=True,
                             mode='min')

In [ ]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=1, min_lr=0.001)
callbacks = [print_callback, checkpoint, reduce_lr]

In [ ]:
model.fit(x, y, batch_size=128, epochs=5, callbacks=callbacks)

In [ ]:
def generate_text(length, diversity):
    # Get random starting text
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    for i in range(length):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char
    return generated

In [ ]:
print(generate_text(500, 0.2)

In [109]:
s= "👍"

In [110]:
s[0]

'👍'